使用 Max Planck Institute for Biogeochemistry 的天气时间序列数据集。

该数据集包含14个不同的特征，例如气温，大气压力和湿度。从2003年开始，每10分钟收集一次。为了提高效率，本文仅使用2009年至2016年之间收集的数据。

In [6]:
import sys 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph.
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score

## keras
from tensorflow_core.python.keras import Input, Model
from tensorflow_core.python.keras.layers import Dropout, Dense, Attention, LSTM

from numpy.matlib import repmat
import pandas as pd
import sklearn
import math

In [7]:
#数据预处理
path = r'C:\Users\zzsgdsg7\Desktop\2021年B题\附件1 监测点A空气质量预报基础数据2.xls'

data = pd.read_excel(path)
data.iloc[:,0] = pd.to_datetime(data.iloc[:,0])
data.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity', 'pressure','wind','direction'], axis='columns',inplace=True)
#print(data.head())

#数据基本处理
modeltime = data['time']
data = data.drop(columns=['place','time'])
data= data.replace('—', np.nan)
data = data.astype('float')
data[ data< 0 ] = np.nan
print(data.head())
data.insert(0,'time',modeltime)#重新插入时间列

#KNN插值

data_knn =data

def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):#判断值是否是空
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out

for indexs in data.columns:
    if indexs =='time':
        continue
    data_knn[indexs] = knn_mean(data_knn[indexs].values,24)

    
data_knn=pd.DataFrame(data_knn).set_index('time')

   so2   no2  pm10  pm2.5    o3   co  temperature  humidity  pressure  wind  \
0  5.0  75.0  61.0   38.0   1.0  1.0         21.1      89.0    1013.0   1.2   
1  5.0  64.0  60.0   41.0  25.0  0.8         20.4      92.0    1012.4   1.1   
2  4.0  60.0  36.0   40.0  25.0  0.8         20.0      93.0    1011.3   1.1   
3  4.0  39.0  35.0   33.0  41.0  0.7         19.8      93.0    1010.4   1.5   
4  4.0  39.0  28.0   39.0  36.0  0.8         19.9      92.0    1010.0   1.5   

   direction  
0      346.5  
1       18.0  
2       65.9  
3       78.4  
4       73.5  


In [8]:
#将时间序列数据转化为监督问题数据
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [10]:
## * Note: 归一化数据范围为0-1.
values =data_knn.values
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(values)
# 数据转为监督学习数据，time-step=1
reframed = series_to_supervised(scaled, 1, 1)
reframed.info()
# 去掉不想预测的列
reframed.drop(reframed.columns[[12,13,14,15,16,17,18,19,20,21]], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19431 entries, 1 to 19431
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   var1(t-1)   19431 non-null  float32
 1   var2(t-1)   19431 non-null  float32
 2   var3(t-1)   19431 non-null  float32
 3   var4(t-1)   19431 non-null  float32
 4   var5(t-1)   19431 non-null  float32
 5   var6(t-1)   19431 non-null  float32
 6   var7(t-1)   19431 non-null  float32
 7   var8(t-1)   19431 non-null  float32
 8   var9(t-1)   19431 non-null  float32
 9   var10(t-1)  19431 non-null  float32
 10  var11(t-1)  19431 non-null  float32
 11  var1(t)     19431 non-null  float32
 12  var2(t)     19431 non-null  float32
 13  var3(t)     19431 non-null  float32
 14  var4(t)     19431 non-null  float32
 15  var5(t)     19431 non-null  float32
 16  var6(t)     19431 non-null  float32
 17  var7(t)     19431 non-null  float32
 18  var8(t)     19431 non-null  float32
 19  var9(t)     19431 non-nul